# Suspect of too low accuracy in Lesion model
Vocabulary: 
- boardcasting: when adding tensor with different dimension, for example input_p (dim = [batch_size, 250]) and bias_p (dim = [1, 250]), bias_p will automatically "boardcast" to (batch_size, 250) during the elementwise addition

When input information have not reach a layer, it will assume the first axis (batch_size) = 1 
however, the batch_size won't matter... since tf.matmul batch_axis is independent with each other...  


In [ ]:
import tensorflow as tf
import pandas as pd
import os, meta, modeling, data_wrangling, evaluate
from importlib import reload
reload(modeling)


def init(code_name):
    cfg = meta.ModelConfig.from_json(
        os.path.join("models", code_name, "model_config.json")
    )
    cfg.output_ticks = 8
    
    model = modeling.MyModel(cfg, batch_size_override=100)
    model.load_weights(cfg.saved_weights_fstring.format(epoch=400))
    # data = data_wrangling.MyData()
    # test = evaluate.TestSet(cfg, model)
    return cfg, model


cfg, model = init("NS_4M_zer01")
train100 = data_wrangling.load_testset('dataset/testsets/train_r100.pkl.gz')



# Load testset and evaluate an epoch
## y_pred[task][output_name][timetick][tensor]

In [ ]:
tasks = ["exp_os"]

y_pred = {}
for task in tasks:
    model.set_active_task(task)
    y_pred[task] = model([train100[modeling.IN_OUT[task][0]]] * cfg.n_timesteps)



## Keys

In [ ]:
print(f"Task keys: {y_pred.keys()}")
print(f"Output keys: {y_pred['exp_os'].keys()}")
print(f"Weight tensors: {[w.name for w in model.weights]}")

In [ ]:
tf.reduce_mean(y_pred['exp_os']['sem'], axis=(1, 2))

In [ ]:
# Input from O to S, more precisely: tf.matmul(hos, w_hos_hs)  
os = y_pred['triangle']['input_hos_hs']

# Input from P to S, more precisely: tf.matmul(hps, w_hps_hs)
ps = y_pred['triangle']['input_hps_hs']

# Input from C to S, more precisely: tf.matmul(css, w_cs)
cs = y_pred['triangle']['input_css_cs']

# Input from S to S, more precisely: tf.matmul(sem, w_ss)
ss = y_pred['triangle']['input_sem_ss']

# Input from O to P, more precisely: tf.matmul(hop, w_hop_hp)
op = y_pred['triangle']['input_hop_hp']


In [ ]:
all_timesteps = list(range(1, cfg.n_timesteps + 1))
all_timesteps[-11:]

In [ ]:
y_pred["exp_os"]['sem'][[0,1],:,:]

In [ ]:
def mean_unit_input(tensor, mask=None):
    """Mask with output target, reduce sum on items (axis1), mean on units (axis2)"""
    if mask is not None:
        tensor = tf.multiply(tensor, mask)
    return tf.reduce_mean(tf.reduce_mean(tensor, axis=-1), axis=-1)

results_dict = {k: mean_unit_input(globals()[k], train100['sem']) for k in ('os', 'ps', 'cs', 'ss')}
results_dict['op'] = mean_unit_input(op, train100['pho'])
dol_df = pd.DataFrame.from_dict(results_dict)
dol_df.plot()

In [ ]:
results_dict = {k: mean_unit_input(globals()[k], 1-train100['sem']) for k in ('os', 'ps', 'cs', 'ss')}
results_dict['op'] = mean_unit_input(op, 1-train100['pho'])
dol_df = pd.DataFrame.from_dict(results_dict)
dol_df.plot(title="zero target")

In [ ]:
bias_s = [w for w in model.weights if w.name == "my_model_1/bias_s:0"]
bias_p = [w for w in model.weights if w.name == "my_model_1/bias_p:0"]

In [ ]:
tf.reduce_mean(bias_s)

In [ ]:
tf.reduce_mean(bias_p)

In [ ]:
tf.

In [ ]:
dfs = [pd.read_csv(os.path.join(cfg.eval_folder, f'train_r1000_{ts}.csv')) for ts in ('triangle', 'exp_ops', 'ort_sem')]
df = pd.concat(dfs, ignore_index=True)


In [ ]:
sdf = df.loc[(df.timetick==df.timetick.max()) & (df.epoch==400) & (df.output_name=="sem")]

In [ ]:
sdf.sample(10)

In [ ]:
alt.Chart(sdf.sample(100)).mark_point().encode(
    x="act1:Q",
    y=alt.Y("act0:Q", scale=alt.Scale(reverse=True)),
    color="task:N",
    detail="word:N",
    column="acc:N",
    tooltip=['word', 'acc', 'sse', 'act0', 'act1' ]
)